In [1]:
# IMDB Dataset
# IMDB.com 사이트에 있는 영화, 드라마 리뷰(영어)를
# 우리가 모델로 학습할 수 있도옥 이미 단어사전도 만들고, 숫자로 다 바꿔서
# 우리한테 제공하는 데이터셋
# Tensorflow를 이용해서 해당 내용을 구현해 보았어요!
# PyTorch로 구현해 보아요!

In [13]:
%reset -f

import numpy as np
import datetime
import torch
import torch.nn as nn
import torch.optim as optim

from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

In [14]:
# PyTorch는 직접 데이터와 모델을 GPU Memory에 이동시켜야 해요!
# 항상 처음에 PyTorch 환경이 GPU를 사용할 수 있는 환경인지 확인!
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [15]:
# 데이터 로딩
# 우리가 사용하는 데이터는 순차데이터(시게열데이터)
# timestep이라는 개념이 들어가요!
# 자연어를 처리하려고 해요! 각각의 단어( = token = timestep)
# 이런 단어(token, timestep)의 연속이 우리의 sample이 되요!
# 원래는 vocabulary(단어사전)이라는 것부터 만들어야 해요!
# 그런데 IMDB Dataset은 이 과정이 이미 진행되어 있어요!
(x_data_train, y_data_train), (x_data_test, y_data_test) = \
imdb.load_data(num_words=500)

x_data_train, x_data_val, y_data_train, y_data_val = \
train_test_split(x_data_train,
                 y_data_train,
                 test_size=0.2,
                 stratify=y_data_train,
                 random_state=42)

In [16]:
# 데이터 전처리
# 1. 길이를 맞춰야 해요!(리뷰길이가 제각각인데 동일 길이로 맞춰야 해요!)
# Padding 처리를 통해서 길이를 맞춰요!
x_data_train_seq = pad_sequences(x_data_train,
                                 maxlen=100)
x_data_val_seq = pad_sequences(x_data_val,
                               maxlen=100)

# 2. one-hot encoding으로 처리
# token값들은 값의 볼륨이 아니에요. 분류값이에요!
x_data_train_onehot = to_categorical(x_data_train_seq,
                                     num_classes=500)
x_data_val_onehot = to_categorical(x_data_val_seq,
                                   num_classes=500)

In [17]:
# PyTorch에서 사용하는 Tensor 형태로 변환
# GPU를 사용하기 위해서 데이터를 이동
x_train_tensor = torch.FloatTensor(x_data_train_onehot).to(device)
y_train_tensor = torch.FloatTensor(y_data_train).to(device)

x_val_tensor = torch.FloatTensor(x_data_val_onehot).to(device)
y_val_tensor = torch.FloatTensor(y_data_val).to(device)

In [18]:
# 이렇게 만든 데이터를 조금 편하게 사용해 볼거에요!
# batch 처리하기 편하도록 DataLoader를 이용할거에요!
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True)
val_loader = DataLoader(val_dataset,
                        batch_size=64)

In [ ]:
# 데이터 처리가 끝났으니 Model을 구현해보면 되요!
# 1. class를 정의해야 해요! -> 특정 class를 상속해서 만들어야 해요!
# 2. class내에 우리가 사용할 layer를 속성으로 정의해야 해요!
#    => __init__() 안에서 구현
# 3. 순전파 기능을 함수로 구현해야 해요!
#    => forward() 안에서 구현

# nn.Module를 상속해서 class를 생성
class SimpleRNNModel(nn.Module): 
    # input_size : RNN Model의 input 차원 => vocabulary의 크기
    # hidde_size : RNN Layer안의 neuron(node) 수
    def __init__(self, input_size, hidden_size):
        super().__init__() # 상위 class의 초기화 함수 호출
        # PyTorch가 제공하는 RNN Layer
        # batch_first=True는 무슨 의미인가요?
        # 기본적인 형태는 (seq_len, batch_size, input_size) 인데
        # (batch_size, seq_len, input_size) 이 형태가 필요해요!
        self.rnn = nn.RNN(input_size=input_size,
                          hidden_size=hidden_size,
                          batch_first=True)
        # hidden_size : 입력데이터 개수
        # 1 : 출력데이터 개수
        self.fc = nn.Linear(hidden_size, 1)
    
    # 데이터를 어떻게 전달해서 순전파 연산을 수행할건지 기술
    def forward(self, x):
        # out : 모든 시점의 hidden state
        out, _ = self.rnn(x)
        out = out[:, -1, :] # 마지막 시점의 hidden state
        out = self.fc(out)
        return torch.sigmoid(out)
    
# 이렇게 기능이 정의된 모델 클래스로부터
# 실제로 사용할 수 있는 모델을 만들어요!
model = SimpleRNNModel(input_size=500,
                       hidden_size=8).to(device)

In [20]:
# Loss, Optimizer가 있어야 해요!
criterion = nn.BCELoss() # 'binary_crossentropy'
optimizer = optim.Adam(model.parameters(),
                       lr=1e-3)

# Tensorboard
log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
writer = SummaryWriter(log_dir=log_dir)

In [ ]:
# 학습
# fit() 함수를 사용하지 않아요!
# 작접 Loop를 돌면서 수동으로 처리!
# 참고로 Lighting은 fit()을 이용해서 학습을 진행!

# 20 epochs를 수행할거에요!
epochs = 20
for epoch in range(1, epochs + 1):
    # 학습
    model.train() # PyTorch에서 모델 학습하기 전에는 반드시 이 함수를 호출
                  # 이 작업을 수행해야 나중에 역전파를 진행할 수 없어요!
    train_loss = 0.0
    train_acc = 0.0

    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(x_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward() # backpropagation
        optimizer.step() # weight 값 갱신

        train_loss += loss.item() * 64
        train_acc += ((outputs > 0.5).float() == y_batch).sum().item()
    
    train_loss /= len(train_loader.dataset) # 전체 데이터 개수로 나눠줘요!
    train_acc /= len(train_loader.dataset) # 정확도가 나와요!

    # 검증
    model.eval()
    val_loss = 0.0
    val_acc = 0.0

    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            outputs = model(x_batch).squeeze()
            loss = criterion(outputs, y_batch)

            val_loss += loss.item() * 64
            val_acc += ((outputs > 0.5).float() == y_batch).sum().item()
        
        val_loss /= len(val_loader.dataset)
        val_acc /= len(val_loader.dataset)

    print(f'Epochs : {epoch}/{epochs}') 
    print(f'Train Accuracy : {train_acc} Train Loss : {train_loss}')
    print(f'Validation Accuracy : {val_acc} Validation Loss : {val_loss}')
# Epochs : 20/20
# Train Accuracy : 0.8127 Train Loss : 0.4260636926651001
# Validation Accuracy : 0.7652 Validation Loss : 0.5257220012664795